In [4]:
import pandas as pd
import numpy as np

# 1. Load kedua dataset
# Ganti path sesuai dengan lokasi file Anda
df_dataset2 = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\dataset berita terkini 2\data_compressed.csv.gz', compression='gzip')
df_dataset3 = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\dataset berita terkini 3\politik_merge_compressed.csv.gz', compression='gzip')

# 2. Sesuaikan nama kolom agar konsisten
# Berdasarkan dataset yang Anda upload, dataset2 memiliki kolom 'content' dan 'title'
# Sedangkan dataset3 memiliki kolom 'Content' dan 'Judul'
df_dataset3 = df_dataset3.rename(columns={'Content': 'content', 'Judul': 'title', 'Waktu': 'date'})

# 3. Pilih hanya kolom yang dibutuhkan
df_dataset2 = df_dataset2[['content', 'title', 'date']]
df_dataset3 = df_dataset3[['content', 'title', 'date']]

# # 4. Hapus duplikat dan nilai kosong dari kedua dataset
# df_dataset2 = df_dataset2.drop_duplicates(subset=['content'])
# df_dataset2 = df_dataset2.dropna(subset=['content', 'title'])

# df_dataset3 = df_dataset3.drop_duplicates(subset=['content'])
# df_dataset3 = df_dataset3.dropna(subset=['content', 'title'])

# 5. Ambil 3500 sampel dari masing-masing dataset
sample_size = 3000
df_sample2 = df_dataset2.sample(n=sample_size, random_state=42)
df_sample3 = df_dataset3.sample(n=sample_size, random_state=42)

# 6. Gabungkan kedua sampel
df_combined = pd.concat([df_sample2, df_sample3], ignore_index=True)

# 7. Acak dataset gabungan untuk memastikan data tercampur dengan baik
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# 8. Simpan dataset gabungan ke file CSV
df_combined.to_csv('berita_terkini.csv', index=False)

# 9. Tampilkan informasi dataset
print(f"Jumlah data dari dataset2: {len(df_sample2)}")
print(f"Jumlah data dari dataset3: {len(df_sample3)}")
print(f"Jumlah data pada dataset gabungan: {len(df_combined)}")
print("\nContoh 5 baris data dari dataset gabungan:")
df_combined

Jumlah data dari dataset2: 3000
Jumlah data dari dataset3: 3000
Jumlah data pada dataset gabungan: 6000

Contoh 5 baris data dari dataset gabungan:


,content,title,date
0,"Presiden bertemu dengan timnas U-20 di GBK, Se...",Jokowi: Beberapa Pemain Timnas U-20 Ingin Kuli...,2023-04-01 10:36:35+00
1,Presiden Joko Widodo (Jokowi) melakukan kunjun...,"Kunker ke Lampung, Jokowi Akan Resmikan Bendun...","Senin, 26 Agu 2024 08:53 WIB"
2,"Menteri Koordinator Politik, Hukum, dan Keaman...","Mahfud Buat Satgas Tindak Lanjuti Rp 349 T, Di...",2023-04-10 05:28:58+00
3,JAKARTA - Cara tambah daya listrik hingga 5.50...,Tambah Daya Listrik hingga 5.500 VA dengan Bay...,2023-03-23 02:41:24+00
4,Sekretaris Jenderal PBNU Gus Saifullah Yusuf (...,"Sekjen PBNU: Hubungan Santri dan Polri Baik, S...","Selasa, 03 Sep 2024 12:43 WIB"
...,...,...,...
5995,"TEMPO.CO, Jakarta - Tim Pembela Prabowo-Gibran...","Hari Ini, Kubu Prabowo-Gibran Bakal Hadirkan 1...",4/4/2024
5996,Bakal calon Wakil Gubernur DKI Jakarta Rano Ka...,Rano Karno Ungkap Obrolan dengan Anies di Kant...,"Rabu, 28 Agu 2024 12:16 WIB"
5997,Lembaga Survei Indonesia (LSI) mengungkapkan b...,"Buah Kerja Keras Airin, Unggul di Semua Simula...","Kamis, 22 Agu 2024 11:45 WIB"
5998,"JAKARTA, KOMPAS.com - Jaksa Penuntut Umum (JP...","Jaksa Bocorkan ""Chat"" Gazalba dengan Perempuan...",8/8/2024


In [2]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from indonesian_number_normalizer import create_normalizer
from tqdm import tqdm

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
normalizer = create_normalizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
df = pd.read_csv(r'D:\SKRIPSI\skripsi_2025\fix_dataset\berita_terkini.csv')
df

,content,title,date
0,"Presiden bertemu dengan timnas U-20 di GBK, Se...",Jokowi: Beberapa Pemain Timnas U-20 Ingin Kuli...,2023-04-01 10:36:35+00
1,Presiden Joko Widodo (Jokowi) melakukan kunjun...,"Kunker ke Lampung, Jokowi Akan Resmikan Bendun...","Senin, 26 Agu 2024 08:53 WIB"
2,"Menteri Koordinator Politik, Hukum, dan Keaman...","Mahfud Buat Satgas Tindak Lanjuti Rp 349 T, Di...",2023-04-10 05:28:58+00
3,JAKARTA - Cara tambah daya listrik hingga 5.50...,Tambah Daya Listrik hingga 5.500 VA dengan Bay...,2023-03-23 02:41:24+00
4,Sekretaris Jenderal PBNU Gus Saifullah Yusuf (...,"Sekjen PBNU: Hubungan Santri dan Polri Baik, S...","Selasa, 03 Sep 2024 12:43 WIB"
...,...,...,...
5995,"TEMPO.CO, Jakarta - Tim Pembela Prabowo-Gibran...","Hari Ini, Kubu Prabowo-Gibran Bakal Hadirkan 1...",4/4/2024
5996,Bakal calon Wakil Gubernur DKI Jakarta Rano Ka...,Rano Karno Ungkap Obrolan dengan Anies di Kant...,"Rabu, 28 Agu 2024 12:16 WIB"
5997,Lembaga Survei Indonesia (LSI) mengungkapkan b...,"Buah Kerja Keras Airin, Unggul di Semua Simula...","Kamis, 22 Agu 2024 11:45 WIB"
5998,"JAKARTA, KOMPAS.com - Jaksa Penuntut Umum (JP...","Jaksa Bocorkan ""Chat"" Gazalba dengan Perempuan...",8/8/2024


In [9]:
df = df.drop_duplicates(subset=['content'])
print(f"Dataset setelah dibersihkan memiliki {len(df)} baris")

Dataset setelah dibersihkan memiliki 5951 baris


PRE PROCESSING

In [11]:
def cleaning(teks):
    if not isinstance(teks, str):
        return ""
    teks = re.sub(r'<.*?>', ' ', teks)
    teks = re.sub(r'https?://\S+|www\.\S+', ' ', teks)
    teks = re.sub(r'ADVERTISEMENT.*?CONTENT', ' ', teks, flags=re.IGNORECASE | re.DOTALL)
    teks = re.sub(r'[^\w\s\d]', ' ', teks)
    teks = re.sub(r'\s+', ' ', teks).strip()
    return teks
df['cleaned_text'] = df['content'].fillna('').apply(cleaning)

In [12]:
def case_folding(teks):
    return teks.lower()
df['case_folded_text'] = df['cleaned_text'].apply(case_folding)

In [13]:
def normalisasi(teks):
    # Normalisasi angka (misalnya "100 juta" menjadi "seratus juta")
    try:
        # Gunakan normalize_text dari instance normalizer
        teks = normalizer.normalize_text(teks)
    except Exception as e:
        print(f"Error saat normalisasi angka: {e}")
        pass
    slang_dict = {
        'dgn': 'dengan',
        'tdk': 'tidak',
        'tsb': 'tersebut',
        'utk': 'untuk',
        'spy': 'supaya',
        'krn': 'karena',
        'jg': 'juga',
        'bs': 'bisa',
        'sdh': 'sudah',
        'blm': 'belum',
        'org': 'orang',
        'yg': 'yang',
        'sy': 'saya',
        'dlm': 'dalam',
        'pd': 'pada',
        'dr': 'dari',
        'kmrn': 'kemarin',
        'skrg': 'sekarang',
        'hrs': 'harus',
        'msk': 'masuk',
        'trs': 'terus',
        'tp': 'tapi',
        'kalo': 'kalau',
        'gak': 'tidak',
        'ga': 'tidak',
        'ngga': 'tidak',
        'gk': 'tidak',
        'thn': 'tahun',
        'bln': 'bulan',
        'sblm': 'sebelum',
        'stlh': 'setelah',
        'milyar': 'miliar',
        'trilliun': 'triliun',
        'jt': 'juta',
        'rb': 'ribu',
        '%': 'persen',
        'usd': 'dolar amerika',
        'rupiah': 'rupiah',
        'rp': 'rupiah',
        ',': 'koma',
    }
    
    # Buat pattern regex untuk seluruh kamus sekaligus (lebih efisien)
    pattern = r'\b(' + '|'.join(slang_dict.keys()) + r')\b'
    
    # Fungsi replacement yang menggunakan kamus
    def replace_match(match):
        return slang_dict[match.group(0)]
    
    # Ganti semua kata slang dalam satu kali proses
    return re.sub(pattern, replace_match, teks)
df['normalized_text'] = df['case_folded_text'].apply(normalisasi)

In [14]:
def tokenzing(teks):
    return word_tokenize(teks)
df['tokens'] = df['normalized_text'].apply(tokenzing)

In [15]:
indo_stopwords = set(stopwords.words('indonesian'))
tambahan_stopwords = {
    'ya', 'juga', 'dari', 'di', 'ke', 'pada', 'untuk', 'bagi', 'dan', 'atau', 
    'tapi', 'namun', 'dengan', 'secara', 'oleh', 'karena', 'sehingga', 'agar',
    'sebab', 'jika', 'bila', 'adalah', 'ini', 'itu', 'detik', 'kata', 'dalam',
    'saat', 'akan', 'tidak', 'yang', 'belum', 'sudah', 'telah', 'bisa', 'dapat', 
    'nya', 'pak', 'bu', 'hal', 'pun'
}
indo_stopwords.update(tambahan_stopwords)
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in indo_stopwords]
df['filtered_tokens'] = df['tokens'].apply(remove_stopwords)

In [16]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming(tokens):
    return [stemmer.stem(word) for word in tokens]
df['stemmed_tokens'] = [stemming(tokens) for tokens in tqdm(df['filtered_tokens'], desc="Stemming")]
df['stemmed_text'] = df['stemmed_tokens'].apply(lambda x: ' '.join(x))

Stemming: 100%|██████████| 5951/5951 [1:00:20<00:00,  1.64it/s]


In [17]:
df['processed_content'] = df['stemmed_text']
display(df[['title', 'content', 'processed_content', 'date']].head())

,title,content,processed_content,date
0,Jokowi: Beberapa Pemain Timnas U-20 Ingin Kuli...,"Presiden bertemu dengan timnas U-20 di GBK, Se...",presiden temu timnas u puluh gbk senayan sabtu...,2023-04-01 10:36:35+00
1,"Kunker ke Lampung, Jokowi Akan Resmikan Bendun...",Presiden Joko Widodo (Jokowi) melakukan kunjun...,presiden joko widodo jokowi kunjung kerja lamp...,"Senin, 26 Agu 2024 08:53 WIB"
2,"Mahfud Buat Satgas Tindak Lanjuti Rp 349 T, Di...","Menteri Koordinator Politik, Hukum, dan Keaman...",menteri koordinator politik hukum aman menko p...,2023-04-10 05:28:58+00
3,Tambah Daya Listrik hingga 5.500 VA dengan Bay...,JAKARTA - Cara tambah daya listrik hingga 5.50...,jakarta daya listrik ratus va bayar rpdua ratu...,2023-03-23 02:41:24+00
4,"Sekjen PBNU: Hubungan Santri dan Polri Baik, S...",Sekretaris Jenderal PBNU Gus Saifullah Yusuf (...,sekretaris jenderal pbnu gus saifullah yusuf g...,"Selasa, 03 Sep 2024 12:43 WIB"
